In [1]:
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import requests
import random
import sys
import re

In [2]:
wnids_pc=pd.read_csv('./wnids_parent_child.csv')
wnids_pc=wnids_pc.drop(labels=['Unnamed: 0'],axis=1)
x=pd.read_csv('./words_wnids_gloss_imagenet.csv')
print(x.shape)
xml_dataframe=x.dropna(how='any')
xml_dataframe=xml_dataframe.drop(labels=['Unnamed: 0'],axis=1)
print(wnids_pc.shape,xml_dataframe.shape)
wnids_pc.head(),xml_dataframe.head()

(32296, 4)
(75850, 2) (32295, 3)


(       child     parent
 0  n02119789  n02118333
 1  n02478875  n02471300
 2  n02473983  n02471762
 3  n02100735  n02100399
 4  n02390258  n02374149,
                                                gloss       wnid  \
 0  (botany) a living organism lacking the power o...  n00017222   
 1  photosynthetic or plant constituent of plankto...  n01383896   
 2                                  unicellular algae  n01384084   
 3  microscopic unicellular marine or freshwater c...  n01401106   
 4  microscopic plants; bacteria are often conside...  n11530008   
 
                       words  
 0  plant, flora, plant life  
 1             phytoplankton  
 2          planktonic algae  
 3                    diatom  
 4                microflora  )

In [14]:
#load glove_txt of 100d
#build a dataframe with 'word' and dim-data.
fd=open('./glove.6B\glove.6B.100d.txt','r',encoding='utf-8')
dim=100

word_embeddings=pd.DataFrame()
words_list=[]
rep_list=np.zeros((1,dim))
total_words=0

def update_dataframe(line):
    #first word is word.
    #the rest of them to be converted to float and made to a list.
    global word_embeddings,words_list,rep_list,total_words
    line=line.split()
    words_list.append(line[0])
    rep_row=np.array([float(x) for x in line[1:]]).reshape(1,dim)
    rep_list=np.append(rep_list,rep_row,axis=0)
    if(rep_list.shape[0]%10000==0):
        total_words+=10000
        print(total_words,"Updating csv..")
        word_embeddings=pd.DataFrame(rep_list[1:,:])
        word_embeddings['words']=words_list
        words_list=[]
        rep_list=np.zeros((1,dim))
        word_embeddings.to_csv('./word_embeddings_100d.csv',mode='a', header=False)
    
while(True):
    line=fd.readline()
    if(not line):
        break
    update_dataframe(line)
        

10000 Updating csv..
20000 Updating csv..
30000 Updating csv..
40000 Updating csv..
50000 Updating csv..
60000 Updating csv..
70000 Updating csv..
80000 Updating csv..
90000 Updating csv..
100000 Updating csv..
110000 Updating csv..
120000 Updating csv..
130000 Updating csv..
140000 Updating csv..
150000 Updating csv..
160000 Updating csv..
170000 Updating csv..
180000 Updating csv..
190000 Updating csv..
200000 Updating csv..
210000 Updating csv..
220000 Updating csv..
230000 Updating csv..
240000 Updating csv..
250000 Updating csv..
260000 Updating csv..
270000 Updating csv..
280000 Updating csv..
290000 Updating csv..
300000 Updating csv..
310000 Updating csv..
320000 Updating csv..
330000 Updating csv..
340000 Updating csv..
350000 Updating csv..
360000 Updating csv..
370000 Updating csv..
380000 Updating csv..
390000 Updating csv..
400000 Updating csv..


In [3]:
%%time
#once that is done - get all the gloss vector and 'word' embeddings.
#save with wnids and 'words'
word_embeddings=pd.read_csv('./word_embeddings_100d.csv',header=None)
word_embeddings=word_embeddings.drop(labels=[0],axis=1)

Wall time: 1min


In [4]:
#get the last column - for words.
word_embeddings=word_embeddings.rename(columns={101:'words'})
word_embeddings.head()

,1,2,3,4,5,6,7,8,9,10,...,92,93,94,95,96,97,98,99,100,words
0,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,0.087459,...,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620,the
1,-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,-0.094189,...,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577,","
2,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,-0.019169,...,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803,.
3,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,0.425410,...,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640,of
4,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,0.342410,...,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830,to


In [ ]:
#build representation for each gloss.
#Concatenate the new df to the xml_dataframe
glosses=xml_dataframe['gloss'].tolist()
glosses_arr=np.zeros((1,100))

file_out=open('gloss_conversion.txt','w')

def preprocess(text):
    text=re.sub(r'[^a-zA-Z0-9.\s]','',text)
    return text.split()

def sumup(tokens):
    global word_embeddings
    gloss_rep=np.zeros((1,100))
    for tok in tokens:
        try:
            #print(tok,file=file_out)
            rep=word_embeddings.loc[word_embeddings['words']==tok,word_embeddings.columns!='words'].as_matrix()
            gloss_rep=gloss_rep+rep
        except:
            print(sys.exc_info()[1],file=file_out)
            print("Missing tok.. ",tok,file=file_out)
            
    return gloss_rep/len(tokens) #normalize this to nullify effect of number of tokens in the string.        
i=0
for gloss_ in glosses:
    print(i,gloss_,file=file_out)
    i+=1
    tokens=preprocess(gloss_)
    rep=sumup(tokens) 
    glosses_arr=np.append(glosses_arr,rep,axis=0)
    if(i%10==0):
        print(i,"Written to csv..")
        full_df=pd.DataFrame(glosses_arr[1:,:])
        glosses_arr=np.zeros((1,100))
        full_df.to_csv('./words_wnids_gloss_embeddings.csv',mode='a',header=False)

file_out.close()    
#full_df=pd.concat([xml_dataframe,glosses_df],axis=0)
#print(full_df.shape)

10 Written to csv..
20 Written to csv..
30 Written to csv..
40 Written to csv..
50 Written to csv..
60 Written to csv..
70 Written to csv..
80 Written to csv..
90 Written to csv..
100 Written to csv..
110 Written to csv..
120 Written to csv..
130 Written to csv..
140 Written to csv..
150 Written to csv..
160 Written to csv..
170 Written to csv..
180 Written to csv..
190 Written to csv..
200 Written to csv..
210 Written to csv..
220 Written to csv..
230 Written to csv..
240 Written to csv..
250 Written to csv..
260 Written to csv..
270 Written to csv..
280 Written to csv..
290 Written to csv..
300 Written to csv..
310 Written to csv..
320 Written to csv..
330 Written to csv..
340 Written to csv..
350 Written to csv..
360 Written to csv..
370 Written to csv..
380 Written to csv..
390 Written to csv..
400 Written to csv..
410 Written to csv..
420 Written to csv..
430 Written to csv..
440 Written to csv..
450 Written to csv..
460 Written to csv..
470 Written to csv..
480 Written to csv..
4